In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Generate combine_bc.csv

In [ ]:
import pandas as pd

black_inscope= pd.read_csv('/public/xye6/Xerox/black_inscope.csv',sep='\t',index_col=0)
black_inscope['dim_asset_inscope']=1


black_outscope=pd.read_csv('/public/xye6/Xerox/black_outscope.csv',sep='\t',index_col=0)
black_outscope['dim_asset_inscope']=0

color_inscope=pd.read_csv('/public/xye6/Xerox/color_inscope.csv',sep='\t',index_col=0)
color_inscope['dim_asset_inscope']=1


color_outscope=pd.read_csv('/public/xye6/Xerox/color_outscope.csv',sep='\t',index_col=0)
color_outscope['dim_asset_inscope']=0

In [ ]:
black=pd.concat([black_inscope,black_outscope])
color=pd.concat([color_inscope,color_outscope])

In [ ]:
combine_bc = pd.merge(black, color, how='outer', on=['dim_asset_sk', 'dim_acct_sk','month','dim_asset_inscope'])
combine_bc=combine_bc.sort_values(['dim_acct_sk','dim_asset_sk','month'])

In [ ]:
#combine_bc.to_csv('/public/xye6/Xerox/combine_bc.csv', sep='\t')

### Final processed dataset 
### info_total_with_class.csv

In [ ]:
import pandas as pd
location_info= pd.read_csv('/public/xye6/Xerox/volume_location.csv',sep='\t',index_col=0)
#combine_bc=pd.read_csv('/public/xye6/Xerox/combine_bc.csv',sep='\t',index_col=0)
combine_bc['ColorVolume'].fillna(0,inplace=True)
combine_bc['BlackVolume'].fillna(0,inplace=True)

In [ ]:
info_total=pd.merge(combine_bc,location_info,on=['dim_asset_sk','dim_acct_sk'],how='left')

In [ ]:
def ppm_classification(row):
    
    if row['max_ppm']>=0 and row['max_ppm']<20:
        return 1
        
    elif row['max_ppm']>=20 and row['max_ppm']<40:
        return 2
            
    elif row['max_ppm']>=40 and row['max_ppm']<60:
        return 3
            
    elif row['max_ppm']>=60 and row['max_ppm']<200:
        return 4
            
    else:
        return 5
    

##info_total_with_class.csv
info_total['classification']=info_total.apply(ppm_classification,axis=1)

### ppm_analysis.csv (for tableau volume analysis)

In [ ]:
tmp1=info_total_with_class.groupby(['month','dim_model_manf_nm','dim_asset_inscope']).agg({'BlackVolume':'sum','ColorVolume':'sum'}).reset_index()
tmp2=info_total_with_class[['dim_model_manf_nm','max_ppm','region','classification']].drop_duplicates()


In [ ]:
ppm_analysis=pd.merge(tmp1,tmp2,on='dim_model_manf_nm',how='left')


##ppm_analysis.csv
ppm_analysis['total_volume']=ppm_analysis['BlackVolume']+ppm_analysis['ColorVolume']

### BW_outscope_withppm.csv 
### CL_outscope_withppm.csv

In [ ]:
info_all=pd.read_csv('/public/xye6/Xerox/info_total_with_class.csv')
BW_Acct_with_loc_manuf_slopes=pd.read_csv('/public/ywu82/BW_Acct_with_loc_manuf_slopes.csv')
COL_Acct_with_loc_manuf_slopes=pd.read_csv('/public/ywu82/COL_Acct_with_loc_manuf_slopes.csv')


In [ ]:
BW_outscope=info_all.groupby(['dim_asset_sk','dim_acct_sk','region','dim_model_manf_nm','dim_asset_inscope']).agg({'BlackVolume':'sum'}).reset_index()
CL_outscope=info_all.groupby(['dim_asset_sk','dim_acct_sk','region','dim_model_manf_nm','dim_asset_inscope']).agg({'ColorVolume':'sum'}).reset_index()

BW_outscope=BW_outscope[BW_outscope['dim_asset_inscope']==0]
CL_outscope=CL_outscope[CL_outscope['dim_asset_inscope']==0]

t1=BW_Acct_with_loc_manuf_slopes[['dim_acct_sk','region','dim_model_manf_nm','slopes','p-value']]
t2=COL_Acct_with_loc_manuf_slopes[['dim_acct_sk','region','dim_model_manf_nm','slopes','p-value']]

BW_outscope_with_slope=pd.merge(BW_outscope,t1,on=['dim_acct_sk','region','dim_model_manf_nm'],how='left')
CL_outscope_with_slope=pd.merge(CL_outscope,t2,on=['dim_acct_sk','region','dim_model_manf_nm'],how='left')

BW_outscope_with_slope1=BW_outscope_with_slope[BW_outscope_with_slope['slopes'].notnull()]
CL_outscope_with_slope1=CL_outscope_with_slope[CL_outscope_with_slope['slopes'].notnull()]

t=info_all[['dim_asset_sk','max_ppm']]
BW_outscope_withppm=pd.merge(BW_outscope_with_slope1,t,on=['dim_asset_sk'],how='left')
CL_outscope_withppm=pd.merge(CL_outscope_with_slope1,t,on=['dim_asset_sk'],how='left')

In [ ]:
#BW_outscope_withppm.to_csv('/public/xye6/Xerox/BW_outscope_withppm.csv',index=False)
#CL_outscope_withppm.to_csv('/public/xye6/Xerox/CL_outscope_withppm.csv',index=False)